In [3]:
import ee
import json
import folium
import geopandas as gpd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.keras import metrics

In [4]:
ee.Initialize()

# 1. Get the image we want to make predictions on

In [5]:
# Read in the image from EE
sample_image = ee.Image("projects/esg-satelite/assets/mars/modelling/model_input/test1")

# 2. Convert to TensorFlow record

In [6]:
KERNEL_SIZE = 128

KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
KERNEL_BUFFER = [int(KERNEL_SIZE/2), int(KERNEL_SIZE/2)]

In [7]:
# Create "stack"
image_stack = ee.Image.cat(
    [
        sample_image
    ]
)

In [8]:
image_stack = image_stack.float()

In [9]:
task = ee.batch.Export.image.toCloudStorage(
    image=image_stack,
    description="TensorFlow record of sample area of THEMIS Tempe Terra.",
    bucket="esg-satelite-data-warehouse",
    fileNamePrefix=f"mars/predictions/features/themis_sample_1/themis_sample_1",
    fileFormat = "TFRecord",
    region = ee.Feature(sample_image.geometry()).geometry(),
    scale = 500,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': KERNEL_BUFFER,
      'compressed': True,
    }
)

In [10]:
task.start()

In [11]:
task.status()

{'state': 'READY',
 'description': 'TensorFlow record of sample area of THEMIS Tempe Terra.',
 'creation_timestamp_ms': 1697572624562,
 'update_timestamp_ms': 1697572624562,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'TLUQURVR47J2ACQR467Q2IDC',
 'name': 'projects/earthengine-legacy/operations/TLUQURVR47J2ACQR467Q2IDC'}

## 2.1. Plot on Folium

In [12]:
# Creating add_ee_layer function
# This allows us to visualise EE objects on a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee_image_object.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [13]:
Map = folium.Map() 

Map.add_ee_layer(
    sample_image,
    {},
    "Tempe Terra"
)

Map.add_ee_layer(
    ee.FeatureCollection(sample_image.geometry()),
    {"color": "red"},
    "Geometry"
)

_ = folium.LayerControl().add_to(Map)


In [14]:
Map

# 3. Make prediction 

In [15]:
OUTPUT_FOLDER = "gs://esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1"

In [16]:
BANDS = ["b1"]

In [ ]:
exportFilesList = !gsutil ls {OUTPUT_FOLDER}

imageFilesList = []
jsonFile = None
for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
        imageFilesList.append(f)
    elif f.endswith('.json'):
        jsonFile = f

imageFilesList.sort()

jsonText = !gsutil cat {jsonFile}
mixer = json.loads(jsonText.nlstr)
patches = mixer['totalPatches']

# x_buffer = int(KERNEL_BUFFER[0] / 2)
# y_buffer = int(KERNEL_BUFFER[1] / 2)

buffered_shape = [
    KERNEL_SHAPE[0] + KERNEL_BUFFER[0],
    KERNEL_SHAPE[1] + KERNEL_BUFFER[1]
]

imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) 
    for k in BANDS
]

imageFeaturesDict = dict(zip(BANDS, imageColumns))

def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
imageDataset = imageDataset.map(toTupleImage).batch(1)

In [1]:
model = tf.keras.models.load_model("gs://esg-satelite-data-warehouse/mars/models/sample_model_1")

NameError: name 'tf' is not defined

In [2]:
def plot_results(x_train, x_pred_prob, x_pred_binary, title):
    fig, axs = plt.subplots(1, 3)
    
    axs[0].imshow(x_train)
    _ = axs[0].axis('off')
    _ = axs[0].set_title('Real Image') 
    
    axs[1].imshow(x_pred_prob)
    _ = axs[1].axis('off')
    _ = axs[1].set_title('Predicted Fault (prob)')
    
    axs[2].imshow(x_pred_binary)
    _ = axs[2].axis('off')
    _ = axs[2].set_title('Predicted Fault (bin)')
    
    plt.suptitle(title)
    
def get_model_results(model, data, n_images = 1, thresh = 0.5):
    
    # NOTE: Make sure "reshuffle_each_iteration = False" in data.shuffle(...)
    for i in range(n_images):
        # Get data
        x_train = np.expand_dims(np.concatenate([x for x in data], axis=0)[i], axis=0)
        # Get predictions
        x_pred_prob = model.predict(x_train)
        x_pred_binary = (x_pred_prob > thresh)*1
        
        plot_results(x_train[:,:,:,:1].squeeze(), x_pred_prob[0], x_pred_binary[0], title = f"IOU: N/A") 

In [ ]:
get_model_results(model, imageDataset)